%md
source API URL : "https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2023-01-01&end_date=2024-01-01&daily=temperature_2m_max,temperature_2m_min,rain_sum"

In [0]:
# source_data = 'https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2023-01-01&end_date=2024-01-01&daily=temperature_2m_max,temperature_2m_min,rain_sum'
source_base_url = 'https://archive-api.open-meteo.com/v1/archive?latitude='
source_relative_url = '&start_date=2023-01-01&end_date=2024-01-01&daily=temperature_2m_max,temperature_2m_min,rain_sum'
sink_layer = 'bronze'
sink_folder = 'weather-data'
sink_account_name ='madhupavanadls'
destination_location = f'abfss://{sink_layer}@{sink_account_name}.dfs.core.windows.net/{sink_folder}'

In [0]:
import pandas as pd
import numpy as np
import requests
import json
import datetime
import time

In [0]:
%python
Sql_query = '''
    select distinct latitude, longitude, market_name 
    from uc_prod.silver.geo_location_silver
    where latitude is not null and longitude is not null
    limit 100
'''
lat_long = spark.sql(Sql_query).collect()

weather_data = []

for item in lat_long:
    latitude = item.latitude
    longitude = item.longitude
    market_name = item.market_name

    source_url = source_base_url + str(latitude) + '&longitude=' + str(longitude) + source_relative_url
    response = requests.get(source_url).json()

    if isinstance(response, dict):
        response['market_name'] = market_name
        weather_data.append(response)
pd_df = sc.parallelize(weather_data)
spark_df = spark.read.json(pd_df)


# Save as Delta table
spark_df.filter((col("latitude")!= None ) or(col("latitude")!= '')).write.format('delta').mode('overwrite').option('mergeSchema', 'true').save(destination_location)